In [ ]:
import numpy as np
import cv2

In [ ]:
with_mask = np.load('withmask.npy')
without_mask = np.load('withoutmask.npy')

In [ ]:
with_mask.shape

In [ ]:
without_mask.shape

In [ ]:
with_mask = with_mask.reshape(200,50*50*3)
without_mask = without_mask.reshape(200,50*50*3)

In [ ]:
with_mask.shape

In [ ]:
without_mask.shape

In [ ]:
X = np.r_[with_mask,without_mask]

In [ ]:
X.shape

In [ ]:
labels = np.zeros(X.shape[0])

In [ ]:
labels[200:]=1.0

In [ ]:
names = {0:'Mask',1:'No Mask'}

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,labels, test_size=0.25)

In [ ]:
x_train.shape


In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=3)
x_train = pca.fit_transform(x_train)

In [ ]:
x_train[0]

In [ ]:
x_train.shape

In [ ]:
svm = SVC()
svm.fit(x_train, y_train)

In [ ]:
x_test = pca.fit_transform(x_test)
y_pred = svm.predict(x_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:

haar_data = cv2.CascadeClassifier('data.xml')
capture=cv2.VideoCapture(0)
data = []
font = cv2.FONT_HERSHEY_COMPLEX
while True:
    flag, img = capture.read()
    if flag:
        faces=haar_data.detectMultiScale(img)
        for x,y,w,h in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,255),4)
            face = img[y:y+h,x:x+w, :]
            face = cv2.resize(face,(50,50))
            face = face.reshape(1, -1)
            face = pca.transform(face)
            pred = svm.predict(face)[0]
            n = names[int(pred)]
            cv2.putText(img, n, (x,y), font, 1, (244,250,250), 2)
            print(n)
        if cv2.waitKey(2) == 27:
            break

capture.release()
cv2.destroyAllWindows()